# How to evaluate a RAG application

This example uses [Langchain](https://www.langchain.com) and [Giskard](https://github.com/Giskard-AI/giskard) to evaluate the quality of a RAG application.

In [18]:
#!pip install -r requirements.txt
!pip install "langchain-chroma>=0.1.2"

  Using cached build-1.2.2.post1-py3-none-any.whl.metadata (6.5 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 1.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached pyproject_hooks-1.2.0-py3-none-any.whl.metadata (1.3 kB)
  Using cached requests_oauthlib-2.0.0-py2.py3-none-any.whl.metadata (11 kB)
  Using cached oauthlib-3.2.2-py3-none-any.whl.metadata (7.5 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached asgiref-3.8.1-py3-none-any.whl.metadata (9.3 kB)
  Using cached monotonic-1.6-py2.py3-none-any.whl.metadata (1.5 kB)
  Using cached backoff-2.2.1-py3-none-any.whl.metadata (14 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl.metadata (9.2 kB)
   ━━━━━━━━━━━━━━

In [27]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
MODEL = "gpt-4-turbo"

## Scrape the Website and Split the Content

In [3]:
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)

loader = WebBaseLoader("https://www.ml.school/")
documents = loader.load_and_split(text_splitter)
documents

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://www.ml.school/', 'title': "Building Machine Learning Systems That Don't Suck", 'description': "A live, interactive program that'll help you build production-ready machine learning systems from the ground up.", 'language': 'en'}, page_content='Building Machine Learning Systems That Don\'t Suck"This is the best machine learning course I\'ve done. Worth every cent."Jose Reyes, AI/ML at Cevo AustraliaBuilding Machine Learning Systems That Don\'t SuckA live, interactive program that\'ll help you build production-ready machine learning systems from the ground up.Next cohort:\xa0February 3 - 20, 2025Check the schedule for more details about upcoming cohorts.I want to join!Sign inLearn how to design, build, deploy, and scale machine learning systems to solve real-world problems.I\'ll lose my mind if I see another book or course teaching people the same basic ideas for the hundredth time. Most people are stuck in beginner mode, and finding help to solve re

In [25]:
import pickle

# Save the documents to a file
with open('../../backup/ml-school.pkl', 'wb') as f:
    pickle.dump(documents, f)

## Load the Content in a Vector Store

In [9]:
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=OpenAIEmbeddings()
)

/opt/homebrew/anaconda3/lib/python3.11/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [11]:
embeddings = vectorstore.embeddings
with open('../../backup/embeddings.pkl', 'wb') as f:
    pickle.dump(embeddings, f)

In [28]:
import pickle
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.vectorstores import DocArrayInMemorySearch

# Load the documents from a file
with open('../../backup/documents.pkl', 'rb') as f:
    documents = pickle.load(f)

# Load the embeddings from a file
#with open('embeddings.pkl', 'rb') as f:
    #embeddings = pickle.load(f)

# Reconstruct the vector store
vectorstore = DocArrayInMemorySearch.from_documents(
    documents, embedding=embeddings
)

In [29]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_core.documents import Document

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [30]:
vector_store = Chroma(
    collection_name="default",
    embedding_function=embeddings,
    persist_directory="./chroma_storage"  # This is where the data will be saved
)

In [24]:
vector_store.add_documents(documents)

['845fb055-65fe-45dd-85a6-e85d41fdf445',
 'bbafc25a-cf53-4d2f-ba96-306c074547e1',
 '3e477676-6999-4846-a3d7-ebc44d36d215',
 '40402df6-27e4-4556-aea7-dfcacb3b6057',
 '654cf320-5d7d-4090-9388-cf5c0ebfe8b7',
 '32bfdc14-13c2-4072-b5ea-a416ffd58f51',
 '2f363aba-4b48-4290-b6df-65ce396fdb0a',
 'd488d7b3-e46a-4fea-9e5e-9008c6228250',
 '0cdaeb2b-5e0d-4d3f-9321-49919cb596a5',
 '91b0428e-620d-4f1a-8680-93acafce3109',
 '1ea42bb0-e95a-436e-a391-7ac521012a69']

## Create a Knowledge Base

Let's start by loading the content in a pandas DataFrame.

In [5]:
import pandas as pd

df = pd.DataFrame([d.page_content for d in documents], columns=["text"])
df.head(10)

,text
0,Building Machine Learning Systems That Don't S...
1,program will help you unlearn what you think m...
2,only pay once to join. There are no monthly fe...
3,that make systems work.You are ready to put in...
4,"testing in production, among many others.You'l..."
5,"Wednesdays, we'll host office hours when you c..."
6,as you'd like. No restrictions.Enjoy 18 hours ...
7,to determine how much data you need.The proble...
8,"it with complete confidence.""Juan OlanoMachine..."
9,"learning, beginners will find the sessions go ..."


We can now create a Knowledge Base using the DataFrame we created before.

In [6]:
from giskard.llm.client.litellm import LiteLLMClient
from giskard.rag import KnowledgeBase

knowledge_base = KnowledgeBase(
    df,
    llm_client=LiteLLMClient(model=MODEL)
)

2025-01-11 21:01:08,076 pid:49127 MainThread giskard.llm.embeddings INFO     No embedding model set though giskard.llm.set_embedding_model. Defaulting to openai/text-embedding-3-small since OPENAI_API_KEY is set.


## Generate the Test Set

In [9]:
from giskard.rag import generate_testset

testset = generate_testset(
    knowledge_base,
    num_questions=60,
    agent_description="A chatbot answering questions about the Machine Learning School Website",
)

2025-01-11 20:44:51,013 pid:45906 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.11/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2025-01-11 20:44:53,890 pid:45906 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


Generating questions:   0%|          | 0/60 [00:00<?, ?it/s]

In [7]:
from giskard.rag import QATestset
testset = QATestset.load("../../backup/test-set.jsonl")

In [37]:
!pip install ragas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.9/176.9 kB 2.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.6 MB/s eta 0:00:00


In [38]:
from ragas.testset.persona import generate_personas_from_kg
from ragas.testset.graph import KnowledgeGraph
from ragas.llms import llm_factory


personas = generate_personas_from_kg(kg=kg, llm=llm, num_personas=5)

NameError: name 'kg' is not defined

In [43]:
class Base:
    pass

class Child(Base):
    pass

print(Child.__name__)

Child


Let's display a few samples from the test set.

In [8]:
test_set_df = testset.to_pandas()

for index, row in enumerate(test_set_df.head(3).iterrows()):
    print(f"Question {index + 1}: {row[1]['question']}")
    print(f"Reference answer: {row[1]['reference_answer']}")
    print("Reference context:")
    print(row[1]['reference_context'])
    print("******************", end="\n\n")


Question 1: What do participants receive upon joining the machine learning program?
Reference answer: Participants receive lifetime access to 18 hours of live, interactive sessions, 10 hours of step-by-step coding instructions, a final project with feedback, 100 coding assignments and practice questions, the entire source code of a working production system, a private community, direct access to the instructor, lifetime access to every past and future cohort, and a program certificate upon completion.
Reference context:
Document 1: program will help you unlearn what you think machine learning is. It's a practical, hands-on class where you'll learn from years of experience and real-world examples.When you join, you get lifetime access to the following:18 hours of live, interactive sessions. We'll use this time to discuss the first principles behind building machine learning systems.10 hours of step-by-step coding instructions. These practical sessions will show you how to build an end-t

Let's now save the test set to a file:

In [8]:
testset.save("test-set.jsonl")

## Prepare the Prompt Template

In [33]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Here is some context", question="Here is a question"))


Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: Here is some context

Question: Here is a question



## Create the RAG Chain

Create a retriever from the Vector Store that will allow us to get the top similar documents to a given question.

In [31]:
retriever = vectorstore.as_retriever()
retriever.get_relevant_documents("What is the Machine Learning School?")

/var/folders/xt/mj52lxfd1fd9j29gs0q1pdjc0000gn/T/ipykernel_97959/2472551473.py:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retriever.get_relevant_documents("What is the Machine Learning School?")


[Document(metadata={'source': 'https://www.ml.school/', 'title': "Building Machine Learning Systems That Don't Suck", 'description': "A live, interactive program that'll help you build production-ready machine learning systems from the ground up.", 'language': 'en'}, page_content="program will help you unlearn what you think machine learning is. It's a practical, hands-on class where you'll learn from years of experience and real-world examples.When you join, you get lifetime access to the following:18 hours of live, interactive sessions. We'll use this time to discuss the first principles behind building machine learning systems.10 hours of step-by-step coding instructions. These practical sessions will show you how to build an end-to-end system from scratch.A final project where you'll build a complete solution and receive direct feedback on your work.100 coding assignments and practice questions.The entire source code of a working production system. It's yours. You can change and us

We can now create our chain.

In [32]:
from langchain_openai.chat_models import ChatOpenAI

modelOpenAi = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model=MODEL)

In [12]:
#!pip install -qU langchain_mistralai
#!pip install --upgrade langchain_mistralai
#!pip install --upgrade langchain-core
#!pip install "pydantic<2.0,>=1.10.2"
#!pip show pydantic
#!pip install "pydantic<2.0"

In [13]:
from langchain_mistralai.chat_models import ChatMistralAI
import os

model = ChatMistralAI(
    model="mistral-large-latest",
    temperature=0.7,   # Usually between 0.0 - 1.0
    max_retries=2,
    api_key=os.getenv("MISTRAL_API_KEY"),  # Required in most cases
)

In [18]:
!pip install --upgrade --quiet  langchain-huggingface text-generation transformers google-search-results numexpr langchainhub sentencepiece jinja2 bitsandbytes accelerate

In [26]:
from langchain_huggingface import ChatHuggingFace, HuggingFaceEndpoint
# meta-llama/Llama-3.1-8B-Instruct
llm = HuggingFaceEndpoint(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    task="text-generation",
    max_new_tokens=512,
    do_sample=False,
    repetition_penalty=1.03,
)

chat_model = ChatHuggingFace(llm=llm)

ImportError: tokenizers>=0.21,<0.22 is required for a normal functioning of this module, but found tokenizers==0.20.3.
Try: `pip install transformers -U` or `pip install -e '.[dev]'` if you're working with git main

In [35]:
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

chain = (
        {
            "context": itemgetter("question") | retriever,
            "question": itemgetter("question"),
        }
        | prompt
        | modelOpenAi
        | StrOutputParser()
)

Let's make sure the chain works by testing it with a simple question.

In [36]:
chain.invoke({"question": "What is the Machine Learning School?"})

'The Machine Learning School is a live, interactive program that helps individuals build production-ready machine learning systems from the ground up.'

## Evaluating the Model on the Test Set

We need to create a function that invokes the chain with a specific question and returns the answer.

In [22]:
def answer_fn(question, history=None):
    return chain.invoke({"question": question})

We can now use the `evaluate()` function to evaluate the model on the test set. This function will compare the answers from the chain with the reference answers in the test set.

In [23]:
from giskard.rag import evaluate

report = evaluate(answer_fn, testset=testset, knowledge_base=knowledge_base)

Asking questions to the agent:   0%|          | 0/60 [00:00<?, ?it/s]

CorrectnessMetric evaluation:   0%|          | 0/60 [00:00<?, ?it/s]

Let now display the report.

Here are the five components of our RAG application:

* **Generator**: This is the LLM used in the chain to generate the answers.
* **Retriever**: This is the retriever that fetches relevant documents from the knowledge base according to a query.
* **Rewriter**: This is a component that rewrites the user query to make it more relevant to the knowledge base or to account for chat history.
* **Router**: This is a component that filters the query of the user based on his intentions.
* **Knowledge Base**: This is the set of documents given to the RAG to generate the answers.

In [24]:
display(report)

Loading BokehJS ...

2025-01-11 21:11:55,098 pid:49127 MainThread giskard.rag  INFO     Finding topics in the knowledge base.


/opt/homebrew/anaconda3/lib/python3.11/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/opt/homebrew/anaconda3/lib/python3.11/site-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


2025-01-11 21:11:57,921 pid:49127 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


In [25]:
report.to_html("reports.html")

We can display the correctness results organized by question type.

In [26]:
report.correctness_by_question_type()

,correctness
question_type,
complex,0.6
conversational,0.2
distracting element,0.3
double,0.6
simple,0.8
situational,0.6


We can also display the specific failures.

In [18]:
report.get_failures()

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
adba819f-ca44-400f-9ebf-b43560f269cb,Under what conditions is the next cohort for t...,The next cohort is scheduled for February 3 - ...,Document 0: Building Machine Learning Systems ...,[],"{'question_type': 'complex', 'seed_document_id...",The next cohort for the 'Building Machine Lear...,False,The agent provided the start date and session ...
7f385fe8-0cc6-45d6-9368-cb4cad08667c,Under the condition that participants have no ...,"In Session 1, participants will learn about wh...",Document 6: as you'd like. No restrictions.Enj...,[],"{'question_type': 'complex', 'seed_document_id...",Participants are expected to gain insights int...,False,The agent's answer is missing some details fro...
72c93829-ed80-4a21-9b78-8f060e18b9ef,Could you tell me the exact start date of Coho...,"Cohort 17 starts on February 3, 2025, and ends...","Document 4: testing in production, among many ...",[],"{'question_type': 'complex', 'seed_document_id...",The exact start date of Cohort 17 is February ...,False,The agent provided the start date and schedule...
90ca326a-b30b-41ab-8063-981a7880926e,Upon successfully completing the machine learn...,"Upon completing the machine learning program, ...",Document 1: program will help you unlearn what...,[],"{'question_type': 'complex', 'seed_document_id...",Upon successfully completing the machine learn...,False,The agent provided a list of benefits includin...
3e4121a8-bef5-42ee-93a2-6ccfbed43e25,Could you specify the range of topics discusse...,"The session covers data cleaning, feature engi...",Document 7: to determine how much data you nee...,[],"{'question_type': 'complex', 'seed_document_id...",The topics discussed in the session focused on...,False,The agent's answer is missing some topics such...
ccc21c71-dc4e-4265-867a-0e5956f4c177,Could you provide the duration and schedule fo...,"Cohort 17 runs from February 3 to February 20,...","Document 4: testing in production, among many ...",[],"{'question_type': 'distracting element', 'seed...",Cohort 17 of the program is scheduled to take ...,False,The agent included information about the absen...
51f072ab-c59f-4dcb-911c-8592e079e574,"In Session 1 of the program, are there any spe...",Session 1 covers how to start (almost) any pro...,Document 6: as you'd like. No restrictions.Enj...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,"The agent stated 'I don't know,' but should ha..."
9042cb10-90a8-4271-a1cc-9f7a1a9310c7,Could you elaborate on the sampling strategies...,The sampling strategies discussed are uncertai...,Document 7: to determine how much data you nee...,[],"{'question_type': 'distracting element', 'seed...",I don't know.,False,"The agent stated 'I don't know,' but should ha..."
2a02a2a8-8d02-4958-ab3d-e232ad0caab8,"In the context of the program, does 'lifetime ...",You only pay once to join the program and get ...,"Document 9: learning, beginners will find the ...",[],"{'question_type': 'distracting element', 'seed...","Yes, in the context of the program, 'lifetime ...",False,The agent confirmed that 'lifetime access' inc...


TypeError: RunnableSequence.batch() missing 1 required positional argument: 'inputs'

NotImplementedError: We could not infer your model library. You have two options:
- Pass a prediction_function to the Model class (we will try to serialize it with "cloudpickle").
- Extend the Model class and override the abstract "model_predict" method.
We recommend that you follow our documentation page: https://giskard.readthedocs.io/en/latest/getting-started/scan

In [34]:
from giskard import scan

scan_report = scan(giskard_model)

2025-01-11 21:57:22,176 pid:49127 MainThread LiteLLM      INFO     
LiteLLM completion() model= gpt-4o; provider = openai
2025-01-11 21:57:24,777 pid:49127 MainThread httpx        INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-11 21:57:24,782 pid:49127 MainThread LiteLLM      INFO     Wrapper: Completed Call, calling success_handler
🔎 Running scan…
Estimated calls to your model: ~365
Estimated LLM calls for evaluation: 148

2025-01-11 21:57:25,097 pid:49127 MainThread giskard.scanner.logger INFO     Running detectors: ['LLMBasicSycophancyDetector', 'LLMCharsInjectionDetector', 'LLMHarmfulContentDetector', 'LLMImplausibleOutputDetector', 'LLMInformationDisclosureDetector', 'LLMOutputFormattingDetector', 'LLMPromptInjectionDetector', 'LLMStereotypesDetector', 'LLMFaithfulnessDetector']
Running detector LLMBasicSycophancyDetector…
2025-01-11 21:57:25,118 pid:49127 MainThread LiteLLM      INFO     
LiteLLM completion() model= gpt-4o; provid

KeyboardInterrupt: 

## Creating a Test Suite

We can create a test suite and use it to compare different models.

Load the test set from disk.

In [19]:
from giskard.rag import QATestset

testset = QATestset.load("../../backup/test-set.jsonl")

Create a Test Suite from the test set.

In [20]:
test_suite = testset.to_test_suite("Machine Learning School Test Suite")

We need a function that takes a DataFrame of questions, invokes the chain with each question, and returns the answers.

In [21]:
import giskard


def batch_prediction_fn(df: pd.DataFrame):
    return chain.batch([{"question": q} for q in df["question"].values])

We can now create a Giskard Model object to run our test suite.

In [22]:
giskard_model = giskard.Model(
    model=batch_prediction_fn,
    model_type="text_generation",
    name="Machine Learning School Question and Answer Model",
    description="This model answers questions about the Machine Learning School website.",
    feature_names=["question"], 
)

2025-01-03 18:43:20,499 pid:15163 MainThread giskard.models.automodel INFO     Your 'prediction_function' is successfully wrapped by Giskard's 'PredictionFunctionModel' wrapper class.


Let's now run the test suite using the model we created before.

In [23]:
test_suite_results = test_suite.run(model=giskard_model)

2025-01-03 18:43:20,547 pid:15163 MainThread giskard.datasets.base INFO     Casting dataframe columns from {'question': 'object'} to {'question': 'object'}
2025-01-03 18:43:31,155 pid:15163 MainThread giskard.utils.logging_utils INFO     Predicted dataset with shape (60, 5) executed in 0:00:10.612919
2025-01-03 18:44:09,938 pid:15163 MainThread root         ERROR    An error happened during test execution for test: TestsetCorrectnessTest
Traceback (most recent call last):
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/giskard/core/suite.py", line 522, in run
    result = test_partial.giskard_test(**test_params).execute()
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/anaconda3/lib/python3.11/site-packages/giskard/registry/giskard_test.py", line 195, in execute
    return configured_validate_arguments(self.test_fn)(*self.args, **self.kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt

We can display the results.

In [24]:
display(test_suite_results)

<TestSuiteResult (failed)>

## Integrating with Pytest

In [25]:
import ipytest

We can now integrate our test suite with Pytest.

In [26]:
%%ipytest

import pytest
from giskard.rag import QATestset
from giskard.testing.tests.llm import test_llm_correctness


@pytest.fixture
def dataset():
    testset = QATestset.load("../../backup/test-set.jsonl")
    return testset.to_dataset()


@pytest.fixture
def model():
    return giskard_model


def test_chain(dataset, model):
    test_llm_correctness(model=model, dataset=dataset, threshold=0.5).assert_()

UsageError: Cell magic `%%ipytest` not found.
